# Minimal crowd simulation example (evacuation)

This guide explains how to run multiple simulations with the **CrowdMechanics** library and how to manage their outputs. It has been tested on macOS Sequoia 15.6.1 and Ubuntu 24.04.3 LTS, provided that in what follows the shared library relative path `../../../src/mechanical_layer/build/libCrowdMechanics` is written with the correct extension: `.dylib` on macOS and `.so` on Ubuntu.

---

## 1. Export crowd configuration files

1. Open the [**Streamlit app**](https://lemons.streamlit.app/).
2. Go to the **Crowd** tab.
3. Click **"Initialise your own crowd"**.
4. In the sidebar at the bottom, click **"Export crowd as XML config files"**.
5. Download the XML files and save them in an appropriate location.

---

## 2. Set the working environment

Create two folders in your project directory `static` and `dynamic`. Place the following files that you downloaded from the Streamlit app in the respective folders:
  - Move `Agents.xml`, `Geometry.xml`, and `Materials.xml` into the `static` folder.
  - Move `AgentDynamics.xml` into the `dynamic` folder.

Create a `Parameters.xml` file in your actual folder location. Use the following template (replace the paths with your actual folder locations):

```xml
<?xml version="1.0" encoding="utf-8"?>
<Parameters>
    <Directories Static="./static/" Dynamic="./dynamic/"/>
    <Times TimeStep="0.05" TimeStepMechanical="2e-6"/>
</Parameters>
```

**Directory structure example:**
```
.
├── Parameters.xml
├── static/
|   ├── Agents.xml
│   ├── Geometry.xml
│   └── Materials.xml
└── dynamic/
    └── AgentDynamics.xml
```

---

## 3. Build the C++ project

Navigate to the root of the `mechanical_layer` directory and build the project:

```bash
cmake -H. -Bbuild -DBUILD_SHARED_LIBS=ON
cmake --build build
```

---

## 4. Run a series of simulations

Execute the Python code below, modifying it as needed for your experiments.
Simulation results are automatically saved in the `outputXML/` directory.
Each output file is named as follows:
```
AgentDynamics output t=TIME_VALUE.xml
```
Here, `TIME_VALUE` represents the specific simulation time or identifier for that run.

In [ ]:
import ctypes
import os
import xml.etree.ElementTree as ET
from pathlib import Path
from shutil import copyfile

import numpy as np

# === Simulation Parameters ===
dt = 0.1  # Time step for the decisional layer (matches "TimeStep" in Parameters.xml)
Ndt = 90  # How many dt will be performed in total

# === Paths Setup ===
outputPath = Path("outputXML")  # Directory to store output XML files
inputPath = Path("inputXML")  # Directory to store input XML files
outputPath.mkdir(parents=True, exist_ok=True)  # Create directories if they don't exist
inputPath.mkdir(parents=True, exist_ok=True)
for file in list(outputPath.glob("*.xml")) + list(inputPath.glob("*.xml")):
    os.remove(file) # Clean old XML files

# === Loading the External Mechanics Library ===
# Adjust filename for OS (.so for Linux, .dylib for macOS)
Clibrary = ctypes.CDLL("../../../src/mechanical_layer/build/libCrowdMechanics.dylib")

agentDynamicsFilename = "AgentDynamics.xml"

# Prepare the list of XML files that will be passed to the DLL/shared library
files = [
    b"Parameters.xml",
    b"Materials.xml",
    b"Geometry.xml",
    b"Agents.xml",
    agentDynamicsFilename.encode("ascii"),  # Convert filename to bytes (required by ctypes)
]
nFiles = len(files)  # Number of configuration files to be passed
filesInput = (ctypes.c_char_p * nFiles)()  # Create a ctypes array of string pointers
filesInput[:] = files  # Populate array with the XML file names

# === Main Simulation Loop ===
for t in range(Ndt):
    if t % 10 == 0:
        print("Looping the Crowd mechanics engine - t=%.2fs..." % (t * dt))

    # 1. Save the current AgentDynamics file as input for this step (can be used for analysis later)
    copyfile(
        "dynamic/" + agentDynamicsFilename,
        str(inputPath) + rf"/AgentDynamics input t={t * dt:.2f}.xml",
    )

    # 2. Call the external mechanics engine, passing in the list of required XML files
    if Clibrary.CrowdMechanics(filesInput) != 0:
        print("Crowd mechanics engine failed!")
        exit(0)

    # 3. Save the updated AgentDynamics output to results folder (can be used for analysis later)
    copyfile(
        "dynamic/" + agentDynamicsFilename,
        str(outputPath) + rf"/AgentDynamics output t={(t + 1) * dt:.2f}.xml",
    )

    # 4. If the simulation produced an AgentInteractions.xml file, save that as well (optional output)
    try:
        copyfile(
            "dynamic/AgentInteractions.xml",
            str(outputPath) + rf"/AgentInteractions t={(t + 1) * dt:.2f}.xml",
        )
    except FileNotFoundError:
        # If the AgentInteractions file does not exist, skip copying
        pass

    # === Decision/Controller Layer for Next Step ===
    # Read the output AgentDynamics XML as input for the next run.
    # This is where you (or another program) can set new forces/moments for each agent for the next simulation step.
    XMLtree = ET.parse("dynamic/" + agentDynamicsFilename)
    agentsTree = XMLtree.getroot()

    # -- Assign forces/moments to each agent --
    for agent in agentsTree:
        # Create new <Dynamics> tag for the agent (as the output file doesn't have it)
        dynamicsItem = ET.SubElement(agent, "Dynamics")

        # Assign random force, and random moment
        dynamicsItem.attrib["Fp"] = f"{np.random.normal(loc=200, scale=200):.2f},{np.random.normal(loc=0, scale=50):.2f}"
        dynamicsItem.attrib["Mp"] = f"{np.random.normal(loc=0, scale=5):.2f}"

    # Write the modified XML back, to be used in the next iteration
    XMLtree.write("dynamic/" + agentDynamicsFilename)
    # ================================================

# After all simulation steps are complete, print a final message.
print(f"Loop terminated at t={Ndt * dt:.2f}s!")

Looping the Crowd mechanics engine - t=0.0s...
Looping the Crowd mechanics engine - t=1.0s...
Looping the Crowd mechanics engine - t=2.0s...
Looping the Crowd mechanics engine - t=3.0s...
Looping the Crowd mechanics engine - t=4.0s...
Looping the Crowd mechanics engine - t=5.0s...
Looping the Crowd mechanics engine - t=6.0s...
Looping the Crowd mechanics engine - t=7.0s...
Looping the Crowd mechanics engine - t=8.0s...
Loop terminated at t=9.0s!


For precise explanations, read the `Mechanical layer` tutorial.

---

## 5. Export to ChAOS

The output files can be easily converted into `.csv` format, ready for direct import into [ChAOS](https://project.inria.fr/crowdscience/project/ocsr/chaos/). ChAOS uses these CSV files containing time, x, and y coordinates for each agent, to visualize and animate trajectories for further analysis and video generation.

To get started, install the required Python dependencies by creating a virtual environment with [uv](https://docs.astral.sh/uv/) and syncronizing the dependencies:

```bash
python -m pip install --upgrade pip
pip install uv
uv sync --locked --all-extras --dev
```

You can then run the following Python script:

In [ ]:
import os
from pathlib import Path

from configuration.backup import xml_to_Chaos  # For converting XML to Chaos format

filenameCSV = "all_trajectories.csv"  # Name of the final CSV file we’ll generate
cwd = Path(os.path.abspath(""))  # Current working directory
PathXML = cwd / "inputXML"  # Folder path where the XML files are located
PathCSV = cwd / "inputCSV"  # Folder path where CSV files will be saved
PathCSV.mkdir(parents=True, exist_ok=True)  # Create directories if it doesn't exist
for file in PathCSV.glob("*.csv"):
    os.remove(file) # clean old files

# Convert XML files to a single summarized CSV
xml_to_Chaos.export_XML_to_CSV(PathCSV, PathXML)

# Convert CSV to ChAOS format
xml_to_Chaos.export_CSV_to_CHAOS(PathCSV, dt)

---

## 6. Export to a movie with FFMPEG

A plot of the scene can be generated from each input or output file under `PNG` format using the Python wrapper. 

In [ ]:
import matplotlib.pyplot as plt

import configuration.backup.dict_to_xml_and_reverse as fun_xml  # For converting XML to dictionary and vice versa
from configuration.models.crowd import create_agents_from_dynamic_static_geometry_parameters  # For creating agents based on XML data
from streamlit_app.plot import plot  # For plotting crowd data

# === Prepare the folders ===
# Define the paths to the folders you'll use
outputPath = Path("outputXML")
staticPath = Path("static")
plotsPath = Path("plots")
plotsPath.mkdir(parents=True, exist_ok=True)  # Create plots directory if it doesn't exist

# Remove any old '.png' and '.pdf' files in the plots directory
for file in plotsPath.glob("*.png"):
    os.remove(file)

# === Load static XML files ===
# Read the Agents.xml file as a string and convert it to a dictionary
with open(staticPath / "Agents.xml", encoding="utf-8") as f:
    crowd_xml = f.read()
static_dict = fun_xml.static_xml_to_dict(crowd_xml)

# Read the Geometry.xml file as a string and convert it to a dictionary
with open(staticPath / "Geometry.xml", encoding="utf-8") as f:
    geometry_xml = f.read()
geometry_dict = fun_xml.geometry_xml_to_dict(geometry_xml)

# === Loop over time steps ===
for t in range(Ndt):
    current_time = (t + 1) * dt

    # Check if the dynamics file exists; if not, skip to the next time step
    dynamics_file = outputPath / f"AgentDynamics output t={current_time:.1f}.xml"
    if not dynamics_file.exists():
        print(f"Warning: {dynamics_file} not found, skipping.")
        continue

    # === Read and process the dynamics XML file ===
    # Read the current dynamics XML file as a string and convert it to a dictionary
    with open(dynamics_file, encoding="utf-8") as f:
        dynamic_xml = f.read()
    dynamic_dict = fun_xml.dynamic_xml_to_dict(dynamic_xml)

    # Create a crowd object using the configuration files data
    crowd = create_agents_from_dynamic_static_geometry_parameters(
        static_dict=static_dict,
        dynamic_dict=dynamic_dict,
        geometry_dict=geometry_dict,
    )

    # Plot the crowd and save it to both PNG format
    fig, ax = plot.display_crowd2D(crowd)

    plt.savefig(plotsPath / rf"crowd2D_t={t:d}.png", dpi=300, format="png")
    plt.close()

You can finally combine all the `PNG` images into a video using [FFmpeg](https://ffmpeg.org/). Be sure to update the `FFmpeg` path in the script to match your system. The resulting video will be saved in `.mov` format.

In [ ]:
import subprocess

ffmpeg = "/Users/oscardufour/ffmpeg_bin/ffmpeg"  # To be adjusted to your ffmpeg binary location
movie_name = "evacuation_scenario"
cwd = Path(os.path.abspath(""))
moviesPath = cwd / "movies"
moviesPath.mkdir(parents=True, exist_ok=True)
framerate = int(1.0 / dt)

# 0. Clean the movies folder from old files
for file in list(moviesPath.glob("*.mp4")) + list(moviesPath.glob("*.mov")):
    os.remove(file)

# 1. Create an MP4 movie from PNG images in the plots folder
cmd1 = f"{ffmpeg} -framerate {framerate} -i {plotsPath}/crowd2D_t=%d.png {moviesPath}/{movie_name}.mp4 -loglevel quiet"
subprocess.Popen(cmd1.split()).communicate()

# 2. Convert the MP4 to MOV for compatibility
cmd2 = f"{ffmpeg} -i {moviesPath}/{movie_name}.mp4 -pix_fmt yuv420p {moviesPath}/{movie_name}.mov -loglevel quiet"
subprocess.Popen(cmd2.split()).communicate()

# 3. Remove the intermediate MP4 file
cmd3 = f"rm {moviesPath}/{movie_name}.mp4"
subprocess.Popen(cmd3.split()).communicate()

Here are several snapshots of the video you get:

<tr>
  <td align="center" style="width:100%;">
    <img src="../../../data/images/practical_case_evacuation.png" width="450" alt="An example of a crowd simulation." style="display:block; margin:auto;">
  </td>
</tr>